In [1]:
# Load libraries
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import math
import sys
from scipy.stats import spearmanr

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
impIterative = IterativeImputer(missing_values=np.nan, sample_posterior=True, imputation_order='random')

from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [2]:
import time
start = time.time()
full_dataset = pd.read_pickle("./full_dataset.pkl")
end = time.time()
print(f'Elapsed Time: {(end-start)} seconds')

Elapsed Time: 17.04186773300171 seconds


In [3]:
full_dataset

,CHILDID,PARENTID,S1_ID,S2_ID,S3_ID,S4_ID,S5_ST_ID,S5_ID,S6_ST_ID,S6_ID,...,C7CPTS81,C7CPTS82,C7CPTS83,C7CPTS84,C7CPTS85,C7CPTS86,C7CPTS87,C7CPTS88,C7CPTS89,C7CPTS90
0,0001001C,0001001P,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0001002C,0001002P,1.0,1.0,NaN,6005.0,6005.0,6005.0,6005.0,A053,...,2704.86,2681.98,2700.63,2697.53,2692.50,2665.01,2844.28,2665.86,2764.51,2720.46
2,0001003C,0001003P,1.0,1.0,NaN,9997.0,9997.0,9997.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0001004C,0001004P,1.0,1.0,NaN,6009.0,6009.0,6009.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0001005C,0001005P,1.0,1.0,NaN,3.0,7197.0,7197.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21404,3116024C,3116024P,3116.0,3116.0,NaN,3116.0,7294.0,7294.0,7294.0,3116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21405,5213040C,5213040P,NaN,NaN,NaN,5213.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21406,5245040C,5245040P,NaN,NaN,NaN,5245.0,5245.0,5245.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21407,5245041C,5245041P,NaN,NaN,NaN,5245.0,5245.0,5245.0,5245.0,A290,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Determining the Best Y-Variable

In [4]:
new_dataset = full_dataset.copy()

In [5]:
dtypes = new_dataset.dtypes
columns = new_dataset.columns

In [6]:
for c in columns:
    if (str(dtypes[c]) != 'float64'):
        new_dataset = new_dataset.drop(labels=c, axis=1)

In [7]:
new_dataset

,S1_ID,S2_ID,S3_ID,S4_ID,S5_ST_ID,S5_ID,S6_ST_ID,D2T_ID,D4T_ID,D5T_ID,...,C7CPTS81,C7CPTS82,C7CPTS83,C7CPTS84,C7CPTS85,C7CPTS86,C7CPTS87,C7CPTS88,C7CPTS89,C7CPTS90
0,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,-2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,NaN,6005.0,6005.0,6005.0,6005.0,NaN,-2.0,-2.0,...,2704.86,2681.98,2700.63,2697.53,2692.50,2665.01,2844.28,2665.86,2764.51,2720.46
2,1.0,1.0,NaN,9997.0,9997.0,9997.0,NaN,NaN,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,NaN,6009.0,6009.0,6009.0,NaN,NaN,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,3.0,7197.0,7197.0,NaN,NaN,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21404,3116.0,3116.0,NaN,3116.0,7294.0,7294.0,7294.0,-2.0,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21405,NaN,NaN,NaN,5213.0,NaN,NaN,NaN,NaN,-2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21406,NaN,NaN,NaN,5245.0,5245.0,5245.0,NaN,NaN,-2.0,-2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21407,NaN,NaN,NaN,5245.0,5245.0,5245.0,5245.0,NaN,-2.0,-2.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Finding correlation

In [8]:
def find_correlation(df, column_name):
    df_column = df[column_name]
    correlation_dict = {}
    
    for col in new_dataset.columns:
        current_column = new_dataset[col]
        correlation = df_column.corr(current_column)
        correlation_dict[col] = correlation_dict.get(col, correlation)
    
    correlation_df = pd.DataFrame.from_dict(correlation_dict, orient='index')
    return correlation_df

In [9]:
def find_highest_correlation(df):
    result_df = df.copy()
    result_df = result_df.T
    
    result_df = result_df.iloc[:, (df >= 0.2).to_numpy()]
    
    return result_df.round(3)

In [10]:
C6_Math_corr = find_correlation(new_dataset, 'C6R4MPF')
C7_Math_corr = find_correlation(new_dataset, 'C7R4MPF')

In [11]:
C6_Math_corr = find_highest_correlation(C6_Math_corr)
C7_Math_corr = find_highest_correlation(C7_Math_corr)

In [12]:
C6_Math_corr

,C1R4MNOR,C1R4MSCL,C2R4MNOR,C2R4MSCL,C2R4MTSC,C2R4MPB3,C3R4MNOR,C3R4MSCL,C3R4MTSC,C4R4RSCL,...,T4KEXPLN,T4KCLSSF,T4KSORTS,T4KORDER,T4KRELAT,T4KSOLVE,T4KGRAPH,T4KMEASU,T4KSTRAT,T4KTTRMT
0,0.23,0.221,0.233,0.236,0.225,0.209,0.241,0.248,0.234,0.208,...,0.275,0.312,0.306,0.294,0.28,0.253,0.309,0.219,0.293,0.263


In [13]:
C7_Math_corr

,C3R4MNOR,C3R4MSCL,C3R4MTSC,C5R4MSCL,C5R4MTSC,C6R4MNR5,C6R4MSCL,C6R4MTSC,C7R4MNR8,C7R4MSCL,...,C7R4MPB4,C7R4MPB5,C7R4MPB6,C7R4MPB7,C7R4MPB8,C7R4MPB9,C7R4MSET,C7MTHFLG,A4KMINMT,A4KSUPPO
0,0.219,0.224,0.221,0.21,0.21,0.212,0.226,0.23,0.235,0.324,...,0.24,0.255,0.292,0.321,0.335,0.371,0.243,0.24,0.204,0.218


### Remove the columns with the most NA values

In [14]:
# def remove_highest_empties(df):
#     half_of_df = len(df) / 2
#     column_names = []
    
#     for column_name, _ in df.iteritems():
#         if (df[column_name].isna().sum() > half_of_df):
#             column_names.append(column_name)
    
#     df = df.drop(columns=column_names)
#     return df

In [15]:
# new_dataset = remove_highest_empties(new_dataset)
# new_dataset

### Remove the NA values from the Y-Variable

##### Using 8th grade Math Scores as the Y-variable

In [16]:
# clean_df = new_dataset.copy().dropna(subset=["C6R4MPF"])
# clean_df

### Remove the variable relating to 5th Grade and Above

In [17]:
# def remove_unusable_variables(df):
#     unusable_columns = []
    
#     for column_name, _ in df.iteritems():
#         if ('5' in column_name) or ('6' in column_name) or ('7' in column_name) or ('8' in column_name):
#             if (column_name != 'C6R4MPF'):
#                 unusable_columns.append(column_name)
    
#     df = df.drop(columns=unusable_columns)
#     return df

In [18]:
# clean_df = remove_unusable_variables(clean_df)
# clean_df

### Determining the Correlation Between Values

In [19]:
# def find_correlation(df):
#     return df.corr()
# find the correlation between every column and the test column

In [20]:
# corr_dict = find_correlation(clean_df)
# corr_dict

In [21]:
# mask = (corr_dict['C6R4MPF'] > 0.3)
# mask

In [22]:
# corr_dict['C6R4MPF'][mask]

In [23]:
# corr_dict['C6R4MPF'].loc[((corr_dict['C6R4MPF'] > 0.3) | (corr_dict['C6R4MPF'] < -0.3))] # || < -0.8

In [24]:
# mask.index.values

In [25]:
# def find_covariance(df):
#     return df.cov()

In [26]:
# cov_df = find_covariance(clean_df)
# cov_df

## Creating a new dataset with the columns we're using

In [27]:
# dataset = original_dataset[["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP","C6R4MPF"]]

In [28]:
# dataset = clean_df.copy()

In [29]:
# dataset.shape

In [30]:
# Number of Non-NA values in each column
# dataset.count()

In [31]:
# Number of NA values in each column
# dataset.isna().sum()

In [32]:
# dataset.groupby('C6R4MPF').size()

In [33]:
# cleandf=dataset.copy().dropna(subset=["C6R4MPF"])

In [34]:
# cleandf.shape

In [35]:
# cleandf.isna().sum()

In [36]:
# print(cleandf.groupby('C6R4MPF').size())

In [37]:
# dataset= cleandf[cleandf["C6R4MPF"]>"4"]
# print(dataset.groupby('C6R4MPF').size())

In [38]:
# dataset.shape

In [39]:
# Group some categories together
# conditions = [
#     (dataset['C6R4MPF'] >= "5.0") & (dataset['C6R4MPF'] < "7.0"),
#     (dataset['C6R4MPF'] >= "7.0")& (dataset['C6R4MPF'] <= "9.0")]
# choices = ['C - Low', 'A - High']
# scores = np.select(conditions, choices)
# dataset.loc[:, ('Score')] = scores
# dataset

In [40]:
#remove the C6R4MPF column, now that we have the 'Score' category 
# dataset=dataset.drop('C6R4MPF',1)
# dataset.head()

In [41]:
# print(dataset.groupby('Score').size())

In [42]:
# dataset2=dataset[dataset['Score']=='A - High'].copy()

In [43]:
# dataset2=dataset2.append(dataset[dataset['Score']=='C - Low'].sample(n=4777))

In [44]:
# print(dataset2.groupby('Score').size())

In [45]:
# print(dataset2.groupby('Score').size())

In [46]:
# sample_size = math.floor(0.75*len(dataset2))
# sample_size

In [47]:
# Creating our sample of random lines
# traindf=dataset2.sample(n=sample_size)
# #Listing what rows our sample is using
# traindf.index

In [48]:
# Printing training dataset which imputation will be performed on
# traindf

In [49]:
# Defining our validation data as rows in dataset2 that are not in our training data
# validationdf=dataset2.drop(traindf.index)
# len(validationdf)

In [50]:
# taking out all NA's from validation dataset
# validationdf=validationdf.dropna()
# len(validationdf)

In [51]:
# Do imputation on ALL columns of TRAIN 
# X = traindf[["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP"]]

In [52]:
# X

In [53]:
# data_imputed=pd.DataFrame(impIterative.fit_transform(X),columns=["P4CMPEDU","P5CMPEDU","P6CMPEDU","C4R4MPF","C5R4MPF","W1INCCAT","W5INCCAT","W8INCCAT","WKMOMED","WKDADED","P1HMEMP","P1HDEMP","P7HMEMP","P7HDEMP"])

In [54]:
# data_imputed

In [55]:
# len(data_imputed)

In [56]:
#Create a new column in data_imputed that doesn't exist yet to tack on the y-variable
# data_imputed["Score"]=traindf['Score'].values

In [57]:
# data_imputed

In [58]:
# data_imputed.shape

In [59]:
# print(data_imputed.groupby('Score').size())

In [60]:
# Split-out train dataset
# array = traindf.values
# X_train = array[:,0:14]
# Y_train= array[:,14]
# X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

In [61]:
# Spot Check Algorithms
# models = []
# models.append(('LR', LogisticRegression(max_iter=10000, multi_class='multinomial')))
# models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski', weights='distance')))
# models.append(('CART', DecisionTreeClassifier()))
# models.append(('NB', GaussianNB()))
# models.append(('SVM', SVC(gamma='scale')))
# models.append(('RF', RandomForestClassifier(criterion='gini', n_estimators=10000, n_jobs=2)))

In [62]:
# # evaluate each model in turn
# results = []
# names = []
# for name, model in models:
# 	kfold = StratifiedKFold(n_splits=7, random_state=1, shuffle=True)
# 	cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
# 	results.append(cv_results)
# 	names.append(name)
# 	print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [63]:
# #Set validation set
# X_validation=validationdf.iloc[:,0:14]
# Y_validation=validationdf.iloc[:,14]

In [64]:
# # Make predictions on validation dataset with SVM
 
# model = SVC(gamma='scale')
# model.fit(X_train, Y_train)
# predictions = model.predict(X_validation)

In [65]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions))
# plot_confusion_matrix(model.fit(X_train, Y_train),X_validation,Y_validation,cmap=plt.cm.Blues)
# print(classification_report(Y_validation, predictions))

In [66]:
# # Make predictions on validation dataset with RF
 
# model2 = RandomForestClassifier(criterion='gini', n_estimators=10000, n_jobs=2)
# model2.fit(X_train, Y_train)
# predictions2 = model2.predict(X_validation)

In [67]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions2))
# plot_confusion_matrix(model2.fit(X_train, Y_train),X_validation,Y_validation,cmap=plt.cm.Blues)
# print(classification_report(Y_validation, predictions2))

In [68]:
# feature_importance = model2.feature_importances_

# sorted_idx = np.argsort(feature_importance)[::-1]
# print("Feature Importance:\n")
# for i in range(len(X.columns)):
#     idx = sorted_idx[i]
#     print(f'{X.columns[idx]:20} {feature_importance[idx]:.3f}')